In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.sparse as sp


In [4]:
import torch
from torch import nn
import QuantLib as ql
from scipy.optimize import differential_evolution

#### Attempt 1
failure from the lack of advanced knowledge of numerical methods.

In [5]:
# Description: Crank-Nicholson Scheme for 2D Heat Equation





# Crank-Nicholson Scheme


# Parameters
width = 1.0     # lower-end of the rod
height = 1.0    # upper-end of the rod
N = 100      # number of width divisions
K = 100    # number of height divisions
dx = width / N  # spatial step size
dy = height / K
C = 1.0 # thermal diffusivity
dt = 0.01   # time step size
# crank-nicolson constant
i_c = 0.5 
theta = i_c * C*dt/(dx**2)
alpha = (1 - i_c) * C * dt /(dx**2)
T = 0.1     # total time
M = int(T / dt)  # number of time steps

# Spatial grid
params = np.linspace(0, width, N)
y = np.linspace(0, height, K)
X,Y = np.meshgrid(params, y)
u = np.sin(np.pi * X)* np.sin(np.pi * Y)

# diagonal length is defined by the number of grid points
i_grid_size = (K-1) * (N-1)
# create the coefficient matrix
# diagonal of a 1-4*Lambda 

offdiagonals = np.zeros(i_grid_size - 1)
offdiagonals[(np.arange(1, i_grid_size)) % (K -1) != 0] = -theta
# np.fill(diagonals[(np.arange(1, i_grid_size)) % (K -1) != 0])

# diagonals.fill(-theta)
diagsVec = [-theta , offdiagonals, 1 + 4 * theta, offdiagonals, -theta]
# diags[:, [0, 1, 3, 4]] = -theta
# diags[:,2] = 1 + 4 * theta
# #zeros from BCs
offsets = [-N+1, -1, 0, 1, N-1]


A = scipy.sparse.diags(diagsVec, offsets, shape=((K-1) * (N-1), (K-1) * (N-1))).toarray()
print(A)





print(u)



# Time iteration
#PUT 3D ARRAY
# u_hist = [u.copy()] # history of u for plotting
u_hist = np.zeros(shape=(M + 1, N, K))
u_hist[0,:,:] = u
for k in range(1, M+1):
    b = np.zeros((K-1) * (N-1))
    # print("u_n: ", u)
    for i in range(1, N-1):
        for i in range(1, K-1): 
            index = (i-1) * (N-1)+ i -1  # Adjusted index calculation for interior points
            # b[index] = alpha * u[i+1, j] + alpha * u[i, j + 1] + (1- 4*alpha) * u[i, j] \
            # + alpha * u[i - 1, j] + alpha * u[i, j -1]
            b[index] = (1 - 4 * alpha) * u[i,i] + alpha * (u[i+1,i] + u[i,i+1] + u[i-1,i] + u[i,i-1])
   
    u_new = sp.linalg.spsolve(A, b)
    u_new_2d = np.zeros_like(u)
    index = 0
    # for i in range(1, N):
    #     for j in range(1, K):
    #         u_new_2d[i, j] = u_new[index]
    #         index += 1


    u_new_2d[1:N, 1:K] = u_new.reshape(N-1, K-1)

    u = u_new_2d
    u_hist[k,:,:] = u_new_2d

u_hist = np.array(u_hist)




[[201. -50.   0. ...   0.   0.   0.]
 [-50. 201. -50. ...   0.   0.   0.]
 [  0. -50. 201. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 201. -50.   0.]
 [  0.   0.   0. ... -50. 201. -50.]
 [  0.   0.   0. ...   0. -50. 201.]]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00666176e-03 2.01230991e-03 ... 2.01230991e-03
  1.00666176e-03 3.88555122e-18]
 [0.00000000e+00 2.01230991e-03 4.02259358e-03 ... 4.02259358e-03
  2.01230991e-03 7.76719002e-18]
 ...
 [0.00000000e+00 2.01230991e-03 4.02259358e-03 ... 4.02259358e-03
  2.01230991e-03 7.76719002e-18]
 [0.00000000e+00 1.00666176e-03 2.01230991e-03 ... 2.01230991e-03
  1.00666176e-03 3.88555122e-18]
 [0.00000000e+00 3.88555122e-18 7.76719002e-18 ... 7.76719002e-18
  3.88555122e-18 1.49975978e-32]]


c:\Users\JoyChang\anaconda3\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


In [6]:
# def findAi(S2, S1, S0, dt, r, sigma, dS_plus, dS_minus, grid_size):
#     Sph = S2 - S1
#     Smh = S1 - S0

#     ai = (sigma**2 * S1**2 / (Sph * dS_plus[:-1]) + r * S1 / dS_plus[:-1]) * dt * 0.5
#     ci = (sigma**2 * S1**2 / (Smh * dS_minus[:-1]) - r * S1 / dS_minus[:-1]) * dt * 0.5
#     ai = np.append(ai, (sigma**2 * S1**2 / (Sph[-1] * dS_plus[-1]) + r * S1 / dS_plus[-1]) * dt * 0.5)
#     ci = np.append(ci, (sigma**2 * S1**2 / (Smh[-1] * dS_minus[-1]) - r * S1 / dS_minus[-1]) * dt * 0.5)
#     bi = -(ai + ci + r * dt)
    
#     return scipy.sparse.diags([ci, bi, ai], [-1, 0, 1], shape=(grid_size, grid_size))


# def approximate_black_scholes(s_grid,  K, T, r, sigma, t_i = 1000, option_type = 'call'):
# 	dt = T / t_i
# 	N = s_grid.shape[0]
# 	v = np.zeros((t_i, N))
# 	v[0, :] = np.maximum(s_grid - K, 0) if option_type == 'call' else np.maximum(K - s_grid, 0)
# 	dS_plus = np.diff(s_grid)  # dS[i] = s_grid[i+1] - s_grid[i]
# 	dS_minus = np.insert(dS_plus[:-1], 0, dS_plus[0]) 

# 	for n in range(t_i - 1):
# 		if option_type == 'call':
# 			v[n + 1, 0] = 0  # Call option value at nearly zero asset price
# 			v[n + 1, -1] = s_grid[-1] - K * np.exp(-r * (T - dt * (n + 1)))
# 		elif option_type == 'put':
# 			v[n + 1, 0] = K * np.exp(-r * (T - dt * (n + 1)))
# 			v[n + 1, -1] = 0  # Put option value at the upper boundary
# 		Ai = findAi(s_grid[2:], s_grid[1:-1], s_grid[: -2], dt, r, sigma, dS_plus, dS_minus, N - 2)
# 		Ai_dense = Ai.toarray()
#         # Use numpy.linalg.solve to solve the linear system
# 		v[n + 1, 1:-1] = np.linalg.solve(Ai_dense, v[n, 1:-1])
		
	
# 	return v

In [7]:
import math

S_0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2
d1 = (math.log(S_0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
d2 = d1 - sigma * math.sqrt(T)

call_option_value = (S_0 * math.exp(-r * T) * (math.erfc(d1 * math.sqrt(2)) / 2) - K * math.exp(-r * T) * (math.erfc(d2 * math.sqrt(2)) / 2))

print("Analytical solution for European call option:", call_option_value)


Analytical solution for European call option: -13.329095232525422


In [8]:
def findAi(S2, S1, S0, dt, r, sigma, dS_plus, dS_minus, grid_size):
    Sph = S2 - S1
    Smh = S1 - S0

    ai = (sigma**2 * S1**2 / (Sph * dS_plus[:-1]) + r * S1 / dS_plus[:-1]) * dt * 0.5
    ci = (sigma**2 * S1**2 / (Smh * dS_minus[:-1]) - r * S1 / dS_minus[:-1]) * dt * 0.5
    ai = np.append(ai, (sigma**2 * S1**2 / (Sph[-1] * dS_plus[-1]) + r * S1 / dS_plus[-1]) * dt * 0.5)
    ci = np.append(ci, (sigma**2 * S1**2 / (Smh[-1] * dS_minus[-1]) - r * S1 / dS_minus[-1]) * dt * 0.5)
    bi = -(ai + ci + r * dt)
    return scipy.sparse.diags([ci, bi, ai], [-1, 0, 1], shape=(grid_size, grid_size))


def approximate_black_scholes(s_grid, K, T, rho, sigma, t_i = 1000, option_type = 'call'):
    dt = T / t_i
    N = s_grid.shape[0]
    v = np.zeros((t_i, N))
    v[0, :] = np.maximum(s_grid - K, 0) if option_type == 'call' else np.maximum(K - s_grid, 0)
    

    for n in range(t_i - 1):
        
        if option_type == 'call':
            v[n + 1, 0] = 0  # Call option value at nearly zero asset price
            v[n + 1, -1] = s_grid[-1] - K * np.exp(-rho * (T - dt * (n + 1)))
        elif option_type == 'put':
            v[n + 1, 0] = K * np.exp(-rho * (T - dt * (n + 1)))
            v[n + 1, -1] = 0  # Put option value at the upper boundary
        dS = np.diff(s_grid)
        Ai = findAi(s_grid[2:], s_grid[1:-1], s_grid[: -2], dt, rho, sigma, dS, N)
        v[n + 1, 1:-1] = sp.linalg.spsolve(Ai, v[n, 1:-1])
        
        
        
    return v[-1, -1]

In [9]:
# Setup the parameters for the test case
S0 = 100
K = 100
T = 1
rho = 0.05
sigma = 0.2
s_grid = np.linspace(50, 150, 100)  # Create a grid for the underlying asset prices
t_i = 1000  # Number of time steps
option_type = 'call'

# Calculate the option price
option_price = approximate_black_scholes(s_grid, K, T, rho, sigma, t_i, option_type)
print(f"Calculated Call Option Price: {option_price}")

TypeError: findAi() missing 1 required positional argument: 'grid_size'

In [ ]:
def option_price(sigma, S, K, T, rho, t_i, option_type):
    s_grid = np.linspace(0, 100, 100)
    return approximate_black_scholes(s_grid, K, T, rho, sigma, t_i, option_type)


In [ ]:
def find_implied_volatility(target_price, s_grid, K, T, rho, t_i, option_type, tol=0.001, max_iter=100):
    sigma_low, sigma_high = 0.01, 2.0  # Initial bounds for volatility
    for i in range(max_iter):
        sigma_mid = (sigma_low + sigma_high) / 2
        mid_price = approximate_black_scholes(s_grid, K, T, rho, sigma_mid, t_i, option_type)
        
        if mid_price < target_price:
            sigma_low = sigma_mid
        else:
            sigma_high = sigma_mid
        
        if abs(mid_price - target_price) < tol:
            return sigma_mid, i+1  # Return the found volatility and iteration count
        
    return sigma_mid, max_iter  # Return the best guess and iteration count if convergence criteria not met

# Example usage:
  # Example market price of the option
s_grid = np.linspace(50, 150, 100)  # Example grid for underlying asset prices, adjust as needed
K = 100  # Strike price
T = 1  # Time to maturity in years
rho = 0.05  # Risk-free interest rate
t_i = 365  # Number of time steps, assuming daily steps over a year for simplicity
option_type = 'call'  # Option type ('call' or 'put')
target_market_price = 50

implied_vol, iterations = find_implied_volatility(target_market_price, s_grid, K, T, rho, t_i, option_type)
print(f"Implied Volatility: {implied_vol:.4f} found in {iterations} iterations")


In [ ]:
def capm(risk_free_rate, beta, market_return):
    return risk_free_rate + beta * (market_return - risk_free_rate)

### Attempt 2: Neural parameter optimization for the heston process

### Neuralized Heston Process
https://stackoverflow.com/questions/64456893/quantlib-python-heston-model-generate-path-get-boost-assertion-failed-px

In [10]:
today = ql.Date(21, 10, 2020)
daycount = ql.Actual360()
calendar = ql.TARGET()
r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.02, daycount))
d_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.01, daycount))
v0, kappa, theta, sigma, rho = 0.03, 10, 0.03, 0.4, 0.3
hs_process = ql.HestonProcess(r_ts, d_ts, ql.QuoteHandle(ql.SimpleQuote(100)), v0, kappa, theta, sigma, rho)


timestep = 50
length = 1.0
rng = ql.GaussianRandomSequenceGenerator(ql.UniformRandomSequenceGenerator(2*timestep, ql.UniformRandomGenerator()))
times = list(ql.TimeGrid(length, timestep))
seq = ql.GaussianMultiPathGenerator(hs_process, times, rng)

path_1 = seq.next()

In [11]:
def heston_predictions(v0, kappa, theta, sigma, rho, risk_free_rate_quote: ql.SimpleQuote):
    today = ql.Date().todaysDate()
    # daycount convention, not the difference in days
    daycount = ql.Actual360()
    calendar = ql.TARGET()

    # Convert SimpleQuote to QuoteHandle for use in FlatForward
    risk_free_rate_handle = ql.QuoteHandle(risk_free_rate_quote)

    # Using FlatForward with QuoteHandle and day count convention
    r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, risk_free_rate_handle, daycount))
    d_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, risk_free_rate_handle, daycount))

    hs_process = ql.HestonProcess(r_ts, d_ts, ql.QuoteHandle(ql.SimpleQuote(100)), v0, kappa, theta, sigma, rho)

    timestep = 50
    length = 0.5  # Simulation length in years
    rng = ql.GaussianRandomSequenceGenerator(ql.UniformRandomSequenceGenerator(2*timestep, ql.UniformRandomGenerator()))
    times = list(ql.TimeGrid(length, timestep))
    seq = ql.GaussianMultiPathGenerator(hs_process, times, rng)

    sample_path = seq.next()
    multi_path = sample_path.value()
    print(type(multi_path[0]), type(multi_path[1]))
    # Extracting asset price and volatility paths
    asset_price_path = torch.tensor([multi_path[0][i] for i in range(len(times))])
    volatility_path = torch.tensor([multi_path[1][i] for i in range(len(times))])

    return asset_price_path, volatility_path


In [12]:
heston_predictions(0.03, 10, 0.03, 0.4, 0.3, ql.SimpleQuote(0.02))

<class 'QuantLib.QuantLib.Path'> <class 'QuantLib.QuantLib.Path'>


(tensor([100.0000,  99.0864,  99.2784, 100.1705,  96.2303,  94.5728,  96.8407,
          96.1655,  97.1993,  99.8300,  98.6234, 105.4523, 102.2115, 104.0598,
          99.6415, 101.5727, 101.2879, 101.0077, 106.2235, 108.6584, 107.9149,
         107.0027, 105.9356, 105.3013, 107.4736, 109.9460, 111.2336, 110.3985,
         110.7462, 115.4014, 115.2520, 116.8513, 118.0406, 117.2273, 119.3657,
         116.0170, 116.8188, 116.5752, 117.9612, 116.8807, 116.0364, 112.5169,
         112.3595, 113.0313, 112.8886, 115.3310, 113.2027, 116.5040, 116.8439,
         118.9375, 117.8755]),
 tensor([0.0300, 0.0327, 0.0327, 0.0487, 0.0314, 0.0304, 0.0367, 0.0466, 0.0551,
         0.0576, 0.0608, 0.0859, 0.0772, 0.0903, 0.0753, 0.0823, 0.0632, 0.0554,
         0.0604, 0.0490, 0.0590, 0.0537, 0.0449, 0.0558, 0.0602, 0.0595, 0.0451,
         0.0481, 0.0367, 0.0416, 0.0457, 0.0674, 0.0717, 0.0697, 0.0611, 0.0708,
         0.0640, 0.0581, 0.0477, 0.0405, 0.0418, 0.0358, 0.0350, 0.0284, 0.0291,
         0.

In [ ]:
# Gather daily historical market data
# daily_market_vols = [np.array([...]) for day in range(num_days)]
# daily_spot_prices = [np.array([...]) for day in range(num_days)]
# daily_strike_prices = [np.array([...]) for day in range(num_days)]
# daily_maturity_dates = [np.array([...]) for day in range(num_days)]

# # Estimate daily Heston model parameters
# daily_heston_parameters = []
# for i in range(num_days):
#     parameters = estimate_historical_parameters(daily_market_vols[i], daily_spot_prices[i], daily_strike_prices[i], daily_maturity_dates[i])
#     daily_heston_parameters.append(parameters)


In [ ]:


def daily_heston_parameters(daily_market_vols, daily_strike_prices, daily_spot_prices, daily_maturity_dates):
	daily_heston_parameter = []
	
		# Estimate Heston model parameters for each day
	parameters = estimate_historical_parameters(daily_market_vols, daily_strike_prices, daily_spot_prices, daily_maturity_dates)
	daily_heston_parameters.append(parameters)
	return daily_heston_parameter
def calculate_hist_parameters(market_vols, spot_prices, strike_prices, maturity_dates, num_days):
	for i in range(num_days):
		daily_parameter = daily_heston_parameters(market_vols[i], spot_prices[i], strike_prices[i], maturity_dates[i])
		daily_heston_parameters.append(daily_parameter)

	return daily_heston_parameters

class heston(nn.Module):
	def __init__(self):
		super(heston, self).__init__()
		self.mu = 0.0
		self.kappa = 0.0
		self.theta = 0.0
		self.xi = 0.0
		self.rho = 0.0

	def forward(self, params):
		self.mu, self.kappa, self.theta, self.xi, self.rho = params
		sp_path, v_path = heston_predictions(self.mu, self.kappa, self.theta, self.xi, self.rho)
		s_mean = sp_path.mean()
		p_mean = sp_path.mean()
		s_dev = sp_path - s_mean
		v_dev = v_path - p_mean

		covariance = (s_dev * v_dev).mean()
		e_x = s_mean
		beta = covariance / s_dev.std()
		return e_x, beta
	

class param_LSTM(nn.Module):
	def __init__(self, device='cpu'):
		super(param_LSTM, self).__init__()
		# utilizing 3 layers of LSTM
		self.lstms = nn.ModuleList([
            nn.LSTM(128, hidden_size=64, batch_first=True),
            nn.LSTM(64, hidden_size=32, batch_first=True),
            nn.LSTM(32, hidden_size=16, batch_first=True)  # Ensure consistency in input_size
        ])
		self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(2)])

		self.device = device
		self.linear = nn.Linear(16, 1)
		self.to(device)
	
	def forward(self,params):
		
		# since we would only need the last output of the LSTM
		
		params = params.to(self.device)
		for i in range(len(self.lstms)):
			params, _ = self.lstms[i](params)
			# apply residual connection
			if i < len(self.dropouts):
				params = self.dropouts[i](params)
		params = params[ -1, :]
		params = self.linear(params)
		
		return params
	
	def train_loop(self, train_h, train_eb, val_h, val_eb):
		# chooses gpu if available, otherwise cpu
		
		# chose the adam optimizer for the gradient descent
		train_h, train_eb = train_h.to(self.device), train_eb.to(self.device)
		val_h, val_eb = val_h.to(self.device), val_eb.to(self.device)
		optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
		# currently using MSE loss function to fit the model ()
		loss_fn = nn.MSELoss()

		# training the model for 100 epochs
		for epoch in range(100):
			# set to training mode
			self.train()

			# zero the gradients otherwise they would accumulate
			optimizer.zero_grad()
			pred = self(train_h)
			loss = loss_fn(pred, train_eb)
			loss.backward()
			optimizer.step()


			# set to evaluation mode
			self.eval()
			with torch.no_grad():  # Inference mode, gradients not computed
				val_pred = self(val_h)
				val_loss = loss_fn(val_pred, val_eb)
			
			print(f'Epoch {epoch} loss: {loss.item()}', f'Validation loss: {val_loss.item()}')


class neuralized_heston(nn.Module):
	def __init__(self, device='cpu'):
		super(neuralized_heston, self).__init__()
		self.heston = heston()
		self.param_LSTM = param_LSTM(device)
		self.device = device
		self.to(device)
	
	def forward(self, params):
		params = self.param_LSTM(params)
		return self.heston(params)